In [1]:
from woocommerce import API
import os
from dotenv import load_dotenv

import re

load_dotenv()

woocommerce_url = os.getenv('URL')
woocommerce_key = os.getenv('KEY')
woocommerce_secret = os.getenv('SECRET')

wc_api = API(
    url=woocommerce_url,
    consumer_key=woocommerce_key,
    consumer_secret=woocommerce_secret,
    version='wc/v3'
)

In [3]:
params = {
    'search': "SAUCONY",
    'status': 'publish',
    'password_protected': False,
    'per_page': 95
}

In [4]:
all_products = []

page = 1
while True:
    params['page'] = page
    products = wc_api.get("products", params=params).json()
    if not products:
        break
    all_products.extend(products)
    page += 1

In [ ]:
skus = [product['sku'] for product in all_products]
skus

In [4]:
sun = [products['name'] for products in all_products]
sun

['SAUCONY Endorphin Speed 3',
 'SAUCONY Peregrine 13',
 'SAUCONY Endorphin Speed 3',
 'SAUCONY Endorphin Pro 3',
 'SAUCONY Endorphin Pro 3',
 'SAUCONY Endorphin Speed 3',
 'Sprava za učvrščivanje mišiča - THIGH MASTER - BS3166',
 'BENCH - Kettle Bell 10 Kg',
 'BENCH - Kettle Bell 8 Kg',
 'BENCH - Kettle Bell 6 Kg',
 'BENCH - Kettle Bell 4 Kg',
 'BENCH Hand Dumbells C 2 Kg',
 'BENCH Balance Bord',
 'Omni 21',
 'SAUCONY Ra Endorphine Singlet',
 'SAUCONY Excursion TR11',
 'Trisuit St 2.0 Front Zip',
 'ARENA Classic Silicone Cap']

In [ ]:
sun68 = [product['name']+'-'+product['short_description']+'-'+
         product['sku'] for product in all_products]
proper_sun68 = [product.replace('-', '_') for product in sun68] 
sun68

In [ ]:
sun68_values = []
for item in sun68:
  text = item.replace(' ', '-').replace('Ž', 'z').replace('š', 's').replace('M', 'm')
  pattern = r'<\/?p>|\n(?!\s*$)'
  cleaned_text = re.sub(pattern, '', text)
  sun68_values.append(cleaned_text)

sun68_keys = [product['sku'] for product in all_products]


In [ ]:
sun68_dict = dict(zip(sun68_keys, sun68_values))
sun68_dict

In [ ]:
for product in all_products[:45]:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        print(attribute)
    print('---', product['short_description'])

In [5]:
saucony_products = []

for product in all_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'Saucony' in attribute.get('options', []):
            saucony_products.append(product)

In [6]:
saucony_products_w_stock = []

for product in saucony_products:
    total_stock = 0
    product_id = product['id']
    variations = wc_api.get(f"products/{product_id}/variations").json()

    for variation in variations:
        stock_quantity = variation['stock_quantity']
        total_stock += stock_quantity if stock_quantity is not None else 0

    if total_stock != 0:
        saucony_products_w_stock.append(product)


In [9]:
filtered = [product['id'] for product in saucony_products_w_stock[12:28]]
filtered

[19959,
 19320,
 19313,
 19274,
 19282,
 19948,
 19342,
 19331,
 19303,
 19267,
 19236,
 19228,
 19218,
 19293,
 19256,
 19246]

In [ ]:
missing_sku = []

for product in arena_products_w_stock:
  short_description = product['short_description']
  pattern = r'^(?!.*mušk.{1})(?!.*žensk.{1})(?!.*dje.{1}).*$'
  attributes = product.get('attributes', [])
  for attribute in attributes:
      gender = [attr for attr in attributes if attr.get('name') == 'Spol']
  if not gender and re.search(pattern, short_description, re.IGNORECASE):
     missing_sku.append(product['sku'])

missing_sku

In [ ]:
for product in arena_products_w_stock:
  attributes = product.get('attributes', [])
  kind = [attr for attr in attributes if attr.get('name') == 'Veličina' and attr['options'] == ['Onesize']]
  if kind:
    print(kind)
    print('---', product['permalink'], product['short_description'])


In [ ]:
saucony_products = []

for product in all_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'Saucony' in attribute.get('options', []):
            saucony_products.append(product)


In [ ]:
saucony_originals_products = []

for product in all_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'Saucony Originals' in attribute.get('options', []):
            saucony_originals_products.append(product)


In [ ]:
saucony_with_stock = []

for product in saucony_originals_products:
    total_stock = 0
    product_id = product['id']
    variations = wc_api.get(f"products/{product_id}/variations").json()

    for variation in variations:
        stock_quantity = variation['stock_quantity']
        total_stock += stock_quantity if stock_quantity is not None else 0

    if total_stock != 0:
        saucony_with_stock.append(product)


In [ ]:
missing = []
for product in saucony_with_stock:
    attributes = product.get('attributes', [])
    kind = [attr for attr in attributes if attr.get('name') == 'Podloga']
    if not kind:
        missing.append(product['sku'])

missing

In [ ]:
for product in saucony_with_stock:
    attributes = product.get('attributes', [])
    kind = [attr['options'] for attr in attributes if attr.get(
        'name') == 'Veličina' and isinstance(attr.get('options', []), list) and len(attr.get('options', [])) == 1]
    if kind == [['ONESIZE']]:
        print(kind, product['permalink'])

In [ ]:
for product in saucony_with_stock:
  attributes = product.get('attributes', [])
  if len(attributes) == 6:
    for attribute in attributes:
      print(attribute)
    print('----', product['permalink'])

In [ ]:
saucony_with_stock[79]

In [ ]:
wc_api.get('products/14034').json()

In [ ]:
saucony_out_of_stock_skus = []

for product in saucony_products:
    total_stock = 0
    product_id = product['id']
    variations = wc_api.get(f"products/{product_id}/variations").json()

    for variation in variations:
        stock_quantity = variation['stock_quantity']
        total_stock += stock_quantity if stock_quantity is not None else 0

    if total_stock == 0:
        saucony_out_of_stock_skus.append(product['sku'])


In [ ]:
saucony_product_skus = [product['sku'] for product in saucony_products]

In [ ]:
len(saucony_product_skus), len(saucony_out_of_stock_skus)

In [ ]:
len(set(saucony_product_skus)), len(set(saucony_out_of_stock_skus))

In [ ]:
from collections import Counter

counter = Counter(saucony_product_skus)

duplicate = [item for item, count in counter.items() if count > 1]
duplicate


In [ ]:
active_saucony_skus = [sku for sku in saucony_product_skus if sku not in saucony_out_of_stock_skus]

In [ ]:
active_saucony_products = [product for product in saucony_products if product['sku'] in active_saucony_skus]

In [ ]:
active_saucony_products_wo_short_description = [product['sku'] for product in active_saucony_products if not product['short_description']]

In [ ]:
active_saucony_products_wo_short_description

In [ ]:
# pattern = r'^(?!.*tenisic.{1})(?=.*mušk.{1}).*$'
# pattern = r'^(?!.*tenisic.{1})(?=.*žensk.{1}).*$'

# pattern = r'mušk.{1,}tenisic.{1,}'
# pattern = r'^(?=.*mušk.{1})(?=.*tenisic.{1}).*$'

# pattern = r'žensk.{1,}tenisic.{1,}'
pattern = r'^(?=.*žensk.{1})(?=.*tenisic.{1}).*$'

missing_skus = []

for product in active_saucony_products:
    short_description = product['short_description']
    if re.search(pattern, short_description, re.IGNORECASE):
        attributes = product.get('attributes', [])
        sizechart_attribute = next((attr for attr in attributes if attr.get('name') == 'sizechart'), None)
        if sizechart_attribute is None or 'SAUWFTW' not in sizechart_attribute.get('options'):
                    missing_skus.append(product['sku'])
                    
missing_skus

In [ ]:
missing_skus = []
for product in active_saucony_products:
  attributes = product.get('attributes', [])
  pattern = r'^(?=.*mušk|.*žensk).*$'
  kind = [attr['options'] for attr in attributes if attr.get('name') == 'Vrsta proizvoda']
  if not kind and re.search(pattern, product['short_description'], re.IGNORECASE):
    print(product['short_description'], kind, product['permalink'])
    print('*'*50)
    missing_skus.append(product['sku'])

missing_skus

In [ ]:
active_saucony_products[14]

In [ ]:
for product in saucony_originals_products:
  attributes = product.get('attributes', [])
  gender = [attr for attr in attributes if attr.get('name') == 'Spol']
  if not gender:
    print(product['sku'], product['short_description'], product['name'])

In [ ]:
on_products = []

for product in all_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'On' in attribute.get('options', []):
            on_products.append(product)


In [ ]:
# pattern = r'^(?!.*tenisic.{1})(?=.*mušk.{1}).*$'
# pattern = r'^(?!.*tenisic.{1})(?=.*žensk.{1}).*$'

# pattern = r'mušk.{1,}tenisic.{1,}'
pattern = r'^(?=.*mušk.{1})(?=.*tenisic.{1}).*$'

# pattern = r'žensk.{1,}tenisic.{1,}'
# pattern = r'^(?=.*žensk.{1})(?=.*tenisic.{1}).*$'

missing_skus = []

for product in on_products:
    short_description = product['short_description']
    if re.search(pattern, short_description, re.IGNORECASE):
        attributes = product.get('attributes', [])
        sizechart_attribute = next(
            (attr for attr in attributes if attr.get('name') == 'sizechart'), None)
        if sizechart_attribute is None or 'ONMFTW' not in sizechart_attribute.get('options'):
            missing_skus.append(product['sku'])

missing_skus


In [ ]:
for product in all_products:
  attributes = product.get('attributes', [])

  attribute = [attr for attr in attributes if attr.get('name') == 'sizechart']
  print(attribute, product['short_description'])

missing_skus = []
for product in saucony_originals_products:
    attributes = product.get('attributes', [])
    sex_found= False
    sizechart_found = False
    for attribute in attributes:
        if attribute.get('name') == 'Spol' and attribute.get('options') == ['Muškarci']:
            sex_found = True
        if attribute.get('name') == 'sizechart' and 'SAUMFTW' not in attribute.get('options'):
            sizechart_found = True
    if sex_found and sizechart_found:
        missing_skus.append(product['sku'])
missing_skus


missing_skus = []
for product in saucony_originals_products:
  attributes = product.get('attributes', [])
  for attribute in attributes:
    if attribute.get('name') == 'sizechart' and attribute.get('options') is None:
      missing_skus.append(product['sku'])
missing_skus

In [ ]:
wrong_male_skus = []
wrong_female_skus = []
wrong_uni_skus = []

no_gender = []
empty_gender = []

for product in saucony_originals_products:
    attributes = product.get('attributes', [])

    gender_attribute = next((attr for attr in attributes if attr.get('name') == 'Spol'), None)
    sizechart_attribute = next((attr for attr in attributes if attr.get('name') == 'sizechart'), None)

    if gender_attribute and sizechart_attribute:
        if gender_attribute.get('options') == ['Muškarci'] and 'SAUMFTW' not in sizechart_attribute.get('options'):
            wrong_male_skus.append(product['sku'])
        elif gender_attribute.get('options') == ['Žene'] and 'SAUWFTW' not in sizechart_attribute.get('options'):
            wrong_female_skus.append(product['sku'])
        elif gender_attribute.get('options') == ['Muškarci', 'Žene'] and 'SAUUFTW' not in sizechart_attribute.get('options'):
            wrong_uni_skus.append(product['sku'])
    elif gender_attribute is None:
        no_gender.append(product['sku'])
    else:
        if gender_attribute.get('options') == ['Muškarci']:
            wrong_male_skus.append(product['sku'])
        elif gender_attribute.get('options') == ['Žene']:
            wrong_female_skus.append(product['sku'])
        elif gender_attribute.get('options') == ['Muškarci', 'Žene']:
            wrong_uni_skus.append(product['sku'])

print(wrong_female_skus)
print(wrong_uni_skus)
print(wrong_male_skus)

In [ ]:
for product in saucony_originals_products:
    attributes = product.get('attributes', [])

    sizechart_attribute = next((attr for attr in attributes if attr.get('name') == 'sizechart'), None)

    if sizechart_attribute is None:
        print(product['name'], product['sku'], product['short_description'])


### Additional 

In [ ]:
products = wc_api.get('products/', params = params).json()

In [ ]:
products = wc_api.get('products', params = params).json()

In [ ]:
attributes = products[0].get('attributes')
for attribute in attributes:
  if attribute['name'] == 'sizechart':
    print('yea')
attributes


In [ ]:
product_sku = 'S20768-30'
product = wc_api.get('products', params={'sku': product_sku}).json()
product

In [ ]:
existing_attributes = product["attributes"]

In [ ]:
new_attribute = {
    'id': 7,
    'name': 'sizechart',
    'position': 5,
    'visible': True,
    'variation': False,
    'options': ['SAUMFTW']
}
modified_attribute = {'id': 2,
    'options': ['Saucony Originals']
}

In [ ]:
for attribute in existing_attributes:
    if attribute['id'] == modified_attribute['id']:
        attribute.update(modified_attribute)

In [ ]:
existing_attributes.append(new_attribute)


In [ ]:
product_id = product['id']

In [ ]:
payload = {"attributes": existing_attributes}

In [ ]:
wc_api.put(f"products/{product_id}", payload).json()

In [ ]:
existing_attributes